In [2]:
# モジュールのインポート
from matplotlib import pyplot
import array
import random

import numpy as np
import matplotlib.pyplot as plt

from math import sqrt

from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools
%matplotlib inline

In [3]:
class Room():
    def __init__(self,n,s,fre_move,w=0,l=0,x=0,y=0):
        self.n = n
        self.s = s
        self.fre_move = fre_move
        self.x = x
        self.y = y
        self.w = w
        self.l = l
        
    def distance(self,b):
        return (abs(self.x-b.x)+abs(self.y-b.y))*self.fre_move[b.n]
    def construction(self):
        S = self.w*self.l
        wall = (self.w+self.l)*2
        return S*1+wall*2
    def penalty(self):
        S = self.w*self.l
        return abs(S-self.s)

In [13]:
Room_1 = Room(0,15,[0,1,1,1,1,1,1,1])
Room_2 = Room(1,15,[1,0,1,1,1,1,1,1])
Room_3 = Room(2,15,[1,1,0,1,1,1,1,1])
Room_4 = Room(3,15,[1,1,1,0,1,1,1,1])
Room_5 = Room(4,15,[1,1,1,1,0,1,1,1])
Room_6 = Room(5,15,[1,1,1,1,1,0,1,1])
Room_7 = Room(6,15,[1,1,1,1,1,1,0,1])
Room_8 = Room(7,15,[1,1,1,1,1,1,1,0])

In [26]:
IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

In [28]:
Rooms = {Room_1,Room_2,Room_3,Room_4,Room_5,Room_6,Room_7,Room_8}

for room in Rooms:
    room.w = random.randint(1,10)
    room.l = random.randint(1,10)
    room.y = random.randint(0,1)
    seq_rooms = [i for _, i in sorted(zip(random.sample(range(0,10),10), Rooms))]
for seq_room in seq_rooms:
    start_point = random.randint(0,10)
    zero = []
    one = []
    if seq_room.y == 0:
        left_point = start_point+sum(one)
        seq_room.x = left_point+seq_room.w/2
        zero.append(left_point+seq_room.w)
    else:
        left_point = start_point+sum(one)
        seq_room.x = left_point+seq_room.w/2
        one.append(left_point+seq_room.w)

creator.create("Fitness", base.Fitness, weights=(-1.0, -1.0, -1.0))
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

C:\Users\msy-t\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\msy-t\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [31]:
toolbox.population(n=100)

[{4, 5, 9, 14},
 {9, 11, 16, 17, 19},
 {0, 2, 3, 5, 8},
 {0, 2, 6, 13, 17},
 {7, 8, 11, 12, 18},
 {3, 10, 12, 17, 18},
 {2, 12, 14, 17, 18},
 {0, 6, 8, 13},
 {2, 10, 11, 17, 18},
 {6, 8, 12, 15, 16},
 {3, 4, 8, 11, 18},
 {2, 11, 18, 19},
 {3, 4, 8, 14, 18},
 {6, 10, 18, 19},
 {1, 12, 13, 16, 18},
 {6, 13, 15, 18},
 {5, 14, 15},
 {1, 4, 9, 12, 17},
 {4, 11, 12, 13, 18},
 {2, 5, 9, 11, 16},
 {1, 8, 14, 15, 16},
 {5, 6, 9, 10, 16},
 {0, 14, 15, 16},
 {4, 6, 10, 18, 19},
 {0, 2, 4, 8, 12},
 {0, 9, 10, 13},
 {1, 10, 16, 19},
 {2, 7, 9, 10},
 {2, 3, 9, 12, 19},
 {0, 1, 8, 19},
 {3, 5, 12, 13},
 {4, 5, 10, 17, 18},
 {9, 11, 14, 19},
 {7, 16, 18, 19},
 {0, 2, 7, 9, 13},
 {5, 6, 7, 8, 18},
 {8, 9, 13, 17},
 {7, 11, 18, 19},
 {2, 4, 10, 19},
 {2, 6, 9, 12, 16},
 {2, 5, 7, 11, 12},
 {4, 8, 11, 18},
 {5, 15, 16, 17, 19},
 {9, 11, 15, 16, 19},
 {5, 7, 8, 17},
 {2, 9, 13, 19},
 {2, 6, 7, 9},
 {0, 7, 8, 19},
 {1, 5, 6, 7, 15},
 {7, 12, 15, 18},
 {3, 10, 11, 13, 16},
 {2, 11, 14, 15, 19},
 {4, 5, 9, 1

In [254]:
def evalplan(up_or_down,start_point,width,length,place_seq):
    for room in Rooms:
        room.w = width[room.n]
        room.l = length[room.n]
        room.y = up_or_down[room.n]
    seq_rooms = [i for _, i in sorted(zip(place_seq, Rooms))]
    for seq_room in seq_rooms:
        zero = []
        one = []
        if seq_room.y == 0:
            left_point = start_point[0]+sum(one)
            seq_room.x = left_point+seq_room.w/2
            zero.append(left_point+seq_room.w)
        else:
            left_point = start_point[0]+sum(one)
            seq_room.x = left_point+seq_room.w/2
            one.append(left_point+seq_room.w)
    distance_cost = []
    construction_cost = []
    penalty_sum = []
    for room in Rooms:
        distance_cost.append(sum([room.distance(i) for i in Rooms]))
        construction_cost.append(room.construction())
        penalty_sum.append(room.penalty())
    dis_c = sum(distance_cost)
    con_c = sum(construction_cost)
    pen_c = sum(penalty_sum)
    return dis_c,con_c,pen_c

In [268]:
# 評価関数、交差、突然変異、選択淘汰
toolbox.register("evaluate", evalplan)
toolbox.register("mate1", tools.cxTwoPoints)
toolbox.register("mate2", tools.cxUniform,indpb=0.05)
toolbox.register("mate3", tools.cxPartialyMatched)
toolbox.register("mutate1", tools.mutFlipBit, indpb=0.05)
toolbox.register("mutate2", tools.mutUniformInt, indpb=0.05,low=0,up=10)
toolbox.register("mutate3", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=5)

In [269]:
MU = 300

pop1 = toolbox.population1(n=MU)
pop2 = toolbox.population2(n=MU)
pop3 = toolbox.population3(n=MU)
pop4 = toolbox.population4(n=MU)
pop5 = toolbox.population5(n=MU)
CXPB, MUTPB ,NGEN = 0.5, 0.2, 50
fitnesses = list(map(lambda i:toolbox.evaluate(i[0],i[1],i[2],i[3],i[4]), zip(pop1,pop2,pop3,pop4,pop5)))
fitnesses1 = []
fitnesses2 = []
fitnesses3 = []
for i in range(len(fitnesses)):
    fitnesses1.append((fitnesses[i][0],))
for i in range(len(fitnesses)):
    fitnesses2.append((fitnesses[i][1],))
for i in range(len(fitnesses)):
    fitnesses3.append((fitnesses[i][2],))
for ind1,ind2,ind3,ind4,ind5, fit1 in zip(pop1,pop2,pop3,pop4,pop5,fitnesses1):
    ind1.fitness.values = fit1
    ind2.fitness.values = fit1
    ind3.fitness.values = fit1
    ind4.fitness.values = fit1
    ind5.fitness.values = fit1
fits = [ind1.fitness.values[0] for ind1 in pop1]

for g in range(NGEN):

    print("-- Generation %i --" % g)

    offspring1 = toolbox.select(pop1, len(pop1))
    offspring2 = toolbox.select(pop2, len(pop2))
    offspring3 = toolbox.select(pop3, len(pop3))
    offspring4 = toolbox.select(pop4, len(pop4))
    offspring5 = toolbox.select(pop5, len(pop5))
    offspring1 = list(map(toolbox.clone, offspring1))
    offspring2 = list(map(toolbox.clone, offspring2))
    offspring3 = list(map(toolbox.clone, offspring3))
    offspring4 = list(map(toolbox.clone, offspring4))
    offspring5 = list(map(toolbox.clone, offspring5))

    for child11, child12, child21, child22, child31, child32, child41, child42, child51, child52 in zip(offspring1[::2], offspring1[1::2], offspring2[::2], offspring2[1::2],offspring3[::2], offspring3[1::2],offspring4[::2], offspring4[1::2],offspring5[::2], offspring5[1::2]):
        if random.random() < CXPB:
            toolbox.mate1(child11, child12)
            toolbox.mate2(child21, child22)
            toolbox.mate2(child31, child32)
            toolbox.mate2(child41, child42)
            toolbox.mate3(child51, child52)
            del child11.fitness.values
            del child12.fitness.values
            del child21.fitness.values
            del child22.fitness.values
            del child31.fitness.values
            del child32.fitness.values
            del child41.fitness.values
            del child42.fitness.values
            del child51.fitness.values
            del child52.fitness.values
    for mutant1, mutant2, mutant3, mutant4, mutant5 in zip(offspring1, offspring2, offspring3, offspring4, offspring5):
        if random.random() < MUTPB:
            toolbox.mutate1(mutant1)
            toolbox.mutate2(mutant2)
            toolbox.mutate2(mutant3)
            toolbox.mutate2(mutant4)
            toolbox.mutate3(mutant5)
            del mutant1.fitness.values
            del mutant2.fitness.values
            del mutant3.fitness.values
            del mutant4.fitness.values
            del mutant5.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

-- Generation 0 --


NameError: name 'offspring' is not defined

In [ ]:
def main():
    MU = 300
    pop1 = toolbox.population1(n=MU)
    pop2 = toolbox.population2(n=MU)
    pop3 = toolbox.population3(n=MU)
    pop4 = toolbox.population4(n=MU)
    pop5 = toolbox.population5(n=MU)
    CXPB, MUTPB ,NGEN = 0.5, 0.2, 50

    print("Start of evolution")

    fitnesses = list(map(toolbox.evaluate, zip(pop1,pop2,pop3,pop4,pop5)))
    for ind1, fit1 in zip(pop1, fitnesses):
        ind1.fitness.values = fit1
    
    fits = [ind.fitness.values[0] for ind in pop]
    log_min = [min(fits)]
    log_max = [max(fits)]
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    log_mean = [mean]
    log_std = [std]
    
    print("  Evaluated %i individuals" % len(pop))
    
    for g in range(NGEN):
        
        
        print("-- Generation %i --" % g)

        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))

        pop[:] = offspring

        fits = [ind.fitness.values[0] for ind in pop]
        
        # 母集団の最小、最大、平均、標準偏差を表示
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        log_min.append(min(fits))
        log_max.append(max(fits))
        log_mean.append(mean)
        log_std.append(std)
        gen = list(range(0, NGEN+1))

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
    print("-- End of (successful) evolution --")
    clear_output()
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    return best_ind, gen, log_min, log_max, log_mean, log_std

In [12]:
h = []
print(sum(h))
for room in Rooms:
    room.w = toolbox.width()[room.n]
    h.append(room.w)

A = toolbox.place_seq()
seq_rooms = [i for _, i in sorted(zip(A, Rooms))]

print(A)
print(seq_rooms[1].n)

0


NameError: name 'Rooms' is not defined

In [43]:
a = [0,0]
b = [1,1]
z = map(lambda x:random.randint(x[0],x[1]),zip(a,b))
random.randint(0,1)
list(z)

[0, 0]